In [28]:
import pandas as pd
import numpy as np
import re

import json

In [42]:
variables = [
    'Attr_ N A M E_ethnicity_signal',
    'Attr_ N A M E_gender_signal', 
    'Var_ A G E',
    'Var_ I M M I G R A T I O N', 
    'Var_ N A M E'
]

In [34]:
claude_3_5 = pd.read_csv(r'claude-3.5\data\results.csv')['Response']
deepseek_chat = pd.read_csv(r'deepseek-chat\data\results.csv')['Response']
gemini_3 = pd.read_csv(r'gemini-3\data\results.csv')['Response']
gemma_2_27b = pd.read_csv(r'gemma-2-27b\data\results.csv')['Response']
gpt_4o = pd.read_csv(r'GPT-4o\data\results.csv')['Response']
gpt_5 = pd.read_csv(r'gpt-5\data\results.csv')['Response']
gpt_oss_120b = pd.read_csv(r'gpt-oss-120b\data\results.csv')['Response']
grok_3_mini = pd.read_csv(r'grok-3-mini\data\results.csv')['Response']
qwen_max = pd.read_csv(r'qwen-max\data\results.csv')['Response']

### Need llama 4, nova micro, and 



In [36]:
s = claude_3_5

def parse_embedded_json(x):
    if pd.isna(x):
        return {}
    x = str(x)

    # Extract the first JSON object between {...}
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}

    json_text = m.group(0)

    # Parse JSON safely
    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        return {}

parsed = s.apply(parse_embedded_json)

# Union of keys across all rows; missing keys => NaN
claude_3_5_responses = pd.json_normalize(parsed)

In [50]:
df = deepseek_chat

def parse_messy_json(x):
    if pd.isna(x):
        return {}
    
    x = str(x)

    # 1) Extract the {...} part
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}
    text = m.group(0)

    # 2) First try normal JSON
    try:
        return json.loads(text)
    except:
        pass

    # 3) Fix common issue: unquoted keys
    # Case 1 Credibility: 4  →  "Case 1 Credibility": 4
    fixed = re.sub(r'([,{]\s*)([A-Za-z0-9\s]+)(\s*:)', r'\1"\2"\3', text)

    try:
        return json.loads(fixed)
    except:
        return {}

parsed = df.apply(parse_messy_json)

scores_df = pd.json_normalize(parsed)

deepseek_chat_responses = pd.concat([df.reset_index(drop=True), scores_df], axis=1)

In [55]:
df = gemini_3

def parse_embedded_json(x):
    if pd.isna(x):
        return {}
    x = str(x)

    # extract {...} and parse
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}
    return json.loads(m.group(0))

parsed = df.apply(parse_embedded_json)
scores_df = pd.json_normalize(parsed)

# optional: ensure numeric
scores_df = scores_df.apply(pd.to_numeric, errors="coerce")

gemini_3_responses = pd.concat([df.reset_index(drop=True), scores_df], axis=1)

In [58]:
df = gemma_2_27b

def parse_unquoted_keys_block(x: str) -> dict:
    if pd.isna(x):
        return {}
    x = str(x)

    # Grab the {...} block
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}
    text = m.group(0)

    # Quote unquoted keys:  Case 1 Credibility: 4  ->  "Case 1 Credibility": 4
    # (works at start of object or after commas)
    text = re.sub(r'([{\s,])\s*([A-Za-z0-9 ]+?)\s*:', r'\1"\2":', text)

    # Now it should be valid JSON
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {}

parsed = df.apply(parse_unquoted_keys_block)

scores_df = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")

gemma_2_27b_responses = pd.concat([df.reset_index(drop=True), scores_df], axis=1)

In [60]:
df = gpt_4o

def parse_unquoted_keys_block(x: str) -> dict:
    if pd.isna(x):
        return {}
    x = str(x)

    # Grab the {...} block
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}
    text = m.group(0)

    # Quote unquoted keys:  Case 1 Credibility: 4  ->  "Case 1 Credibility": 4
    # (works at start of object or after commas)
    text = re.sub(r'([{\s,])\s*([A-Za-z0-9 ]+?)\s*:', r'\1"\2":', text)

    # Now it should be valid JSON
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        return {}

parsed = df.apply(parse_unquoted_keys_block)

scores_df = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")

gpt_4o_responses = pd.concat([df.reset_index(drop=True), scores_df], axis=1)

In [80]:
def parse_loose_json(x):
    if pd.isna(x):
        return {}

    x = str(x).strip()

    # Remove possible markdown fences
    x = x.replace("```json", "").replace("```", "").strip()

    # Extract the first {...} block
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}
    text = m.group(0)

    # Try strict JSON first
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass

    # Repair 1: quote unquoted keys
    # { Case 1 Credibility: 4, ... } -> { "Case 1 Credibility": 4, ... }
    repaired = re.sub(r'([{\s,])([A-Za-z0-9 ]+?)\s*:', r'\1"\2":', text)

    # Repair 2: remove trailing commas before } or ]
    repaired = re.sub(r",\s*([}\]])", r"\1", repaired)

    try:
        return json.loads(repaired)
    except json.JSONDecodeError:
        return {}

parsed = gpt_5.apply(parse_loose_json)

gpt_5_responses = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")

In [76]:
def parse_json_object_from_string(x):
    if pd.isna(x):
        return {}

    x = str(x)

    # Extract the first JSON object in the string
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}

    obj_text = m.group(0)

    try:
        return json.loads(obj_text)
    except json.JSONDecodeError:
        return {}

parsed = gpt_oss_120b.apply(parse_json_object_from_string)

scores_df = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")
gpt_oss_120b_responses = pd.concat([gpt_oss_120b.reset_index(drop=True), scores_df.reset_index(drop=True)], axis=1)

In [84]:
s = grok_3_mini   # parse the same Series you're adding columns to

def parse_json_obj(x):
    if pd.isna(x):
        return {}

    x = str(x).strip()

    # If there is any stray text, extract the {...} block
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}

    try:
        return json.loads(m.group(0))
    except json.JSONDecodeError:
        return {}

parsed = s.apply(parse_json_obj)

grok_3_mini_responses = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")

In [87]:
qwen_max


s = qwen_max  # parse the same column you’ll join back to

def parse_fenced_json(x):
    if pd.isna(x):
        return {}

    x = str(x)

    # remove markdown code fences
    x = x.replace("```json", "").replace("```", "").strip()

    # extract {...} in case there’s any extra text
    m = re.search(r"\{.*\}", x, flags=re.DOTALL)
    if not m:
        return {}

    try:
        return json.loads(m.group(0))
    except json.JSONDecodeError:
        return {}

parsed = s.apply(parse_fenced_json)

qwen_max_responses = pd.json_normalize(parsed).apply(pd.to_numeric, errors="coerce")